<a href="https://colab.research.google.com/github/singhmanas1/showcase/blob/feat%2Fpolars_demo_revised/polars_demo_revised/accelerated_data_processing_examples/polars_gpu_engine_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction to the Polars GPU Engine

Polars is a popular single machine DataFrame library powered by an OLAP Query Engine. Beginning in the v1.3 release, Polars can now leverage NVIDIA GPUs for even higher performance through its GPU engine (powered by RAPIDS cuDF).

Designed to make processing 10-100+ GBs of data feel interactive with just a single GPU, this new engine is built directly into the Polars Lazy API – just pass <font color="#76B900">**engine="gpu"**</font> to the `collect` operation.

The GPU engine fully utilizes the Polars optimizer to ensure efficient execution and minimal memory usage, is compatible with the ecosystem of tools built for Polars, and has graceful CPU fallback for unsupported queries.

This notebook is a short introduction to the Polars GPU engine, powered by cuDF.

In [1]:
!nvidia-smi | head

Thu May  8 21:18:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.124.06             Driver Version: 570.124.06     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H100 PCIe               Off |   00000000:01:00.0 Off |                    0 |
| N/A   53C    P0             89W /  310W |    4559MiB /  81559MiB |      0%      Default |


In [2]:
!lscpu | grep -E "Model|Socket"

Model name:                           Neoverse-N1
Model:                                1
Socket(s):                            -


# Setup

## Installing the Polars GPU Engine in other Systems

Install Polars with GPU Engine by using a feature flag in the standard pip install command  `pip install polars[gpu] --extra-index-url=https://pypi.nvidia.com`.

If you already have polars and need to update it, you can add the `-U` flag...
`!pip install -U polars[gpu] --extra-index-url=https://pypi.nvidia.com`

...BUT the system you are currently on doesn't need it

# Install Other Important Dependencies
To use the built-in data visualization capabilities of Polars, you'll need to install a few additional dependencies.  We'll also install pynvml to help us determine which dataset size to use.

In [3]:
!pip install hvplot jupyter_bokeh holoviews pynvml

## Download the Data

We'll be working with a roughly 22GB dataset of [simulated financial transactions from Kaggle](https://www.kaggle.com/datasets/conorsully1/simulated-transactions) by default. If you're running this notebook on Google Colab using the T4 GPU in the Colab free tier, we'll download a smaller version of this dataset (about 20% of the size) to fit on the relatively weaker CPU and GPU.

We're downloading a copy of this dataset from a GCS bucket hosted by NVIDIA to provide faster download speeds. We'll start by downloading the data. This should take about 30 seconds.

In [4]:
import pynvml
import os 
import urllib.request
pynvml.nvmlInit()
pynvml.nvmlDeviceGetName(pynvml.nvmlDeviceGetHandleByIndex(0))
mem = pynvml.nvmlDeviceGetMemoryInfo(pynvml.nvmlDeviceGetHandleByIndex(0))
mem = mem.total/1e9

top_dir = "./data/"
if not os.path.exists(top_dir):
    print('creating data directory')
    os.system('mkdir ./data')
data_dir = './data/financials/'
if not os.path.exists(data_dir):
    print('creating financials data directory')
    os.system('mkdir ./data/financials/')
    


In [5]:
base_url = 'https://storage.googleapis.com/rapidsai/polars-demo/'
fn = 'transactions.parquet'

if mem < 24: # For smaller GPUs
    dl = 'transactions-t4-20.parquet'
    
    if not os.path.isfile(data_dir+fn):
        print(f'Downloading {base_url+dl} to {data_dir+dl}')
        urllib.request.urlretrieve(base_url+dl, data_dir+dl)
        print(f'Renaming {dl} to {fn}')
        os.rename(data_dir+dl, data_dir+fn)
else: # For larger GPUs
    if not os.path.isfile(data_dir+fn):
        print(f'Downloading {base_url+fn} to {data_dir+fn}')
        urllib.request.urlretrieve(base_url+fn, data_dir+fn)

rd = 'rainfall_data_2010_2020.csv'
if not os.path.isfile(data_dir+rd):
    print(f'Downloading {base_url+rd} to {data_dir+rd}')
    urllib.request.urlretrieve(base_url+rd, data_dir+rd)

## Data License and Terms
As this dataset originates from Kaggle, it's governed by a Kaggle dataset-specific license and terms of use.

> ### CC0 1.0 Universal
> **No Copyright**

>The person who associated a work with this deed has dedicated the work to the public domain by waiving all of his or her rights to the work worldwide under copyright law, including all related and neighboring rights, to the extent allowed by law.
You can copy, modify, distribute and perform the work, even for commercial purposes, all without asking permission. See Other Information below.

> **Other Information**

>In no way are the patent or trademark rights of any person affected by CC0, nor are the rights that other persons may have in the work or in how the work is used, such as publicity or privacy rights.
Unless expressly stated otherwise, the person who associated a work with this deed makes no warranties about the work, and disclaims liability for all uses of the work, to the fullest extent permitted by applicable law.
When using or citing the work, you should not imply endorsement by the author or the affirmer.


# Getting Started

With this dataset, we expect to see some meaningful performance benefits using NVIDIA GPUs for computationally heavy queries and limited benefits for basic queries that are primarily bottlenecked by reading data.

To begin, let's use Polars to read the parquet file and look at the schema and first few rows.

In [6]:
import polars as pl
import hvplot.polars # Using the hvplot as the plotting backend
from polars.testing import assert_frame_equal

In [7]:
pl.__version__

'1.24.0'

In [8]:
transactions = pl.scan_parquet(data_dir+fn) # ./data/financials/transactions.parquet

In [9]:
transactions.collect_schema()

Schema([('CUST_ID', String),
        ('START_DATE', Date),
        ('END_DATE', Date),
        ('TRANS_ID', String),
        ('DATE', Date),
        ('YEAR', Int64),
        ('MONTH', Int64),
        ('DAY', Int64),
        ('EXP_TYPE', String),
        ('AMOUNT', Float64)])

In [10]:
transactions.head(5).collect()

CUST_ID,START_DATE,END_DATE,TRANS_ID,DATE,YEAR,MONTH,DAY,EXP_TYPE,AMOUNT
str,date,date,str,date,i64,i64,i64,str,f64
"""CI6XLYUMQK""",2015-05-01,null,"""T8I9ZB5A6X90UG8""",2015-09-11,2015,9,11,"""Motor/Travel""",20.27
"""CI6XLYUMQK""",2015-05-01,null,"""TZ4JSLS7SC7FO9H""",2017-02-08,2017,2,8,"""Motor/Travel""",12.85
"""CI6XLYUMQK""",2015-05-01,null,"""TTUKRDDJ6B6F42H""",2015-08-01,2015,8,1,"""Housing""",383.8
"""CI6XLYUMQK""",2015-05-01,null,"""TDUHFRUKGPPI6HD""",2019-03-16,2019,3,16,"""Entertainment""",5.72
"""CI6XLYUMQK""",2015-05-01,null,"""T0JBZHBMSVRFMMD""",2015-05-15,2015,5,15,"""Entertainment""",11.06


This dataset has a mix of data types (strings, dates, integers, and floats), some missing values, and it looks like the dates were actually parsed as strings. We'll need to handle that during some of our analysis.

# Our first GPU operation: Total aggregate transaction amount

With Polars, to calculate the total spending in of all transactions we simply `sum` the `AMOUNT` column:

In [11]:
transactions.select(pl.col("AMOUNT").sum()).collect()

AMOUNT
f64
2.1367e10


Looks like we're handling a high total transaction volume. Let's run the same query on the GPU.

To use a GPU, we just need to tell Polars to use the GPU engine when we call `collect` by passing `engine="gpu"` as a parameter.


```python
transactions.select(pl.col("AMOUNT").sum()).collect(engine="gpu")
```

The default configuration is likely the right choice for most workloads, but sometimes we want more control. We can provide more detailed configuration options (such as which device to run on and a variety of other options) by passing a Polars `GPUEngine` object to the `engine` parameter instead.


In this notebook, we'll use `pl.GPUEngine`. The default configuration has transparent CPU fallback for unsupported operations, so if we execute an unsupported query we don't get an error. To prove we're running on the GPU, we'll pass a configured engine object that will raise an error if we can't run the query.

_If you're running with [jupyterlab-nvdashboard](https://developer.nvidia.com/blog/maximize-gpu-performance-with-near-real-time-usage-stats-on-nvdashboard-v0-10/) you should see the GPU Memory and Utilization tick up._

In [12]:
gpu_engine = pl.GPUEngine(
    device=0, # This is the default
    raise_on_fail=True, # Fail loudly if we can't run on the GPU.
)

The very first collection on the GPU will take a couple of seconds. The GPU engine is lazy-loaded so that even if the necessary packages are installed, Polars' fast import times are not affected. Consequently, when we trigger GPU execution for the first time, we load a number of additional packages, and initialize GPU-specific data structures and contexts.

In [13]:
transactions.select(pl.col("AMOUNT").sum()).collect(engine=gpu_engine)

AMOUNT
f64
2.1367e10


We probably don't need a GPU for such a simple operation on this dataset. But when we start doing more complex analysis, the high-bandwidth memory and compute power of GPUs will make things much more interactive.

# More Complex Analysis

While the data is synthetic, it's representative of the kinds of datasets that come up in financial services, retail/e-commerce, consumer internet, and other industries.

With this data, we can see how using GPU-accelerated Polars provides a significant productivity boosts by exploring common business questions like:

1. Which customers have the largest total transctions?
2. What's the per-month transaction amount for each spending category over time?
3. How does weather effect the transactions for each category?

Because this data is synthentic, the results won't be meaningful in terms of business conclusions. Rather, we are motivating _why_ using this GPU engine is useful in terms of perfomance and cost reduction. We'll prove that the CPU and GPU engine get the same results in the first example, but as we move throughout the notebook we'll take this for granted and focus on CPU and GPU timings.

Let's start with our first question.

## Which customers have the largest total transactions?

In [14]:
%%time

res_cpu = (
    transactions
    .group_by("CUST_ID")
    .agg(pl.col("AMOUNT").sum())
    .sort(by="AMOUNT", descending=True)
    .head()
    .collect()
)
res_cpu

CPU times: user 2min 3s, sys: 51.9 s, total: 2min 55s
Wall time: 9.17 s


CUST_ID,AMOUNT
str,f64
"""CP2KXQSX9I""",2.3108e6
"""CGOKEO2EH4""",2.2721e6
"""CZ1KK7E2PK""",2.2409e6
"""CXYJF3GWQU""",2.2389e6
"""CZ0G7ZK6HA""",2.2119e6


In [15]:
%%time

res_gpu = (
    transactions
    .group_by("CUST_ID")
    .agg(pl.col("AMOUNT").sum())
    .sort(by="AMOUNT", descending=True)
    .head()
    .collect(engine=gpu_engine)
)
res_gpu

CPU times: user 501 ms, sys: 443 ms, total: 944 ms
Wall time: 499 ms


CUST_ID,AMOUNT
str,f64
"""CP2KXQSX9I""",2.3108e6
"""CGOKEO2EH4""",2.2721e6
"""CZ1KK7E2PK""",2.2409e6
"""CXYJF3GWQU""",2.2389e6
"""CZ0G7ZK6HA""",2.2119e6


In [16]:
assert_frame_equal(res_cpu, res_gpu)

Great! We see a nice performance gain when using the GPU engine and we've confirmed the results are equal.

In addition to the Dataframe interface, Polars also has an SQL interface. We can also use this with the GPU engine, since Polars translates both the DataFrame and SQL interfaces into a query execution plan.

In [17]:
query = """
SELECT CUST_ID, SUM(AMOUNT) as sum_amt
FROM transactions
GROUP BY CUST_ID
ORDER BY sum_amt desc
LIMIT 5
"""

%time pl.sql(query).collect()
%time pl.sql(query).collect(engine=gpu_engine)

CPU times: user 2min 6s, sys: 48.3 s, total: 2min 54s
Wall time: 9.3 s
CPU times: user 460 ms, sys: 463 ms, total: 923 ms
Wall time: 375 ms


CUST_ID,sum_amt
str,f64
"""CP2KXQSX9I""",2.3108e6
"""CGOKEO2EH4""",2.2721e6
"""CZ1KK7E2PK""",2.2409e6
"""CXYJF3GWQU""",2.2389e6
"""CZ0G7ZK6HA""",2.2119e6


## Which customers have the largest single transaction?

Customer `CP2KXQSX9I` had the largest total transactions over time (on the full dataset), but they might not have the largest single transaction. Let's find the top customers by single transaction amount.

In [18]:
%%time

(
    transactions
    .group_by("CUST_ID")
    .agg(pl.col("AMOUNT").max().alias("max_amount"))
    .sort(by="max_amount", descending=True)
    .head()
    .collect()
)

CPU times: user 2min 14s, sys: 1min 5s, total: 3min 19s
Wall time: 9.47 s


CUST_ID,max_amount
str,f64
"""CN8B3RIGIR""",6519.61
"""CTEC3WBHJW""",6516.54
"""C2JMNNNTQQ""",6515.89
"""CZ0G7ZK6HA""",6515.43
"""CAPBVURSHP""",6513.99


In [19]:
%%time

(
    transactions
    .group_by("CUST_ID")
    .agg(pl.col("AMOUNT").max().alias("max_amount"))
    .sort(by="max_amount", descending=True)
    .head()
    .collect(engine=gpu_engine)
)

CPU times: user 503 ms, sys: 288 ms, total: 791 ms
Wall time: 400 ms


CUST_ID,max_amount
str,f64
"""CN8B3RIGIR""",6519.61
"""CTEC3WBHJW""",6516.54
"""C2JMNNNTQQ""",6515.89
"""CZ0G7ZK6HA""",6515.43
"""CAPBVURSHP""",6513.99


Again, we see a nice speedup using the GPU engine.

## Investigating a specific customer

Let's explore the transactions of a single customer now. What was `CIP0I11MG2`'s largest transaction?

In [20]:
%%time

(
    transactions
    .filter(pl.col("CUST_ID") == "CIP0I11MG2")
    .select(pl.col("AMOUNT").max())
    .collect()
)

CPU times: user 5.27 s, sys: 203 ms, total: 5.48 s
Wall time: 80.5 ms


AMOUNT
f64
6201.45


In [21]:
%%time

(
    transactions
    .filter(pl.col("CUST_ID") == "CIP0I11MG2")
    .select(pl.col("AMOUNT").max())
    .collect(engine=gpu_engine)
)

CPU times: user 442 ms, sys: 333 ms, total: 775 ms
Wall time: 341 ms


AMOUNT
f64
6201.45


### Sidebar: Why was performance using the CPU and GPU engines similar for this query?

What we've seen so far is that, for many common queries, we see significant performance increases when we use the GPU. But for this last one, things were more similar.

Using Polars on a GPU isn't **always** a surefire winner in terms of speed compared to Polars on a CPU. For simple queries that aren't computationally heavy, like the last query, we're often limited by the speed that we can read results from disk.

Let's confirm this by profiling.

In [22]:
res, prof = (
    transactions
    .filter(pl.col("CUST_ID") == "CIP0I11MG2")
    .select(pl.col("AMOUNT").max())
    .profile()
)

prof.with_columns(
    ((pl.col("end") - pl.col("start")) / pl.col("end").max() * 100)
    .alias("pct_time_spent")
)

node,start,end,pct_time_spent
str,u64,u64,f64
"""optimization""",0,5,0.006546
"""parquet(./data/financials/tran…",5,76365,99.963345
"""select(AMOUNT)""",76372,76388,0.020946


We spent 99.9%+ of the time just reading the data. Polars can use the GPU-accelerated Parquet reader to read this data, but ultimately when we're **IO bound** like this, there's less opportunity for GPU-acceleration.

Let's try an even more computationally intense query.

## What's the per-month transaction amount for each category over time?

For this query, we'll group and sort down to the individual month, which takes more work.

In [23]:
%%time

res = (
    transactions
    .group_by(["EXP_TYPE", "YEAR", "MONTH"])
    .agg(pl.mean("AMOUNT"))
    .sort(["EXP_TYPE", "YEAR", "MONTH"])
    .collect()
)

CPU times: user 3min 4s, sys: 2min 36s, total: 5min 41s
Wall time: 11.9 s


In [24]:
%%time

res = (
    transactions
    .group_by(["EXP_TYPE", "YEAR", "MONTH"])
    .agg(pl.mean("AMOUNT"))
    .sort(["EXP_TYPE", "YEAR", "MONTH"])
    .collect(engine=gpu_engine)
)

CPU times: user 738 ms, sys: 400 ms, total: 1.14 s
Wall time: 520 ms


Again, since this query does more work we see strong performance benefits on the GPU.

## Interoperability

As mentioned in the introduction, all results will be materialized as standard CPU Polars DataFrames.

That means that any results from GPU-accelerated queries interoperate seamlessly with the rest of the polars ecosystem. For example, we can use Polars' built-in plotting functionality on the result we created from the GPU engine.

**Colab Note**: Using HoloViews/Bokeh in Colab is not fully supported, so plots may not render correctly.

In [25]:
res = res.with_columns(
    pl.datetime(pl.col("YEAR"), pl.col("MONTH"), day=1)
    .alias("year-mon")
)

res.hvplot.scatter(x="year-mon", y="AMOUNT", by="EXP_TYPE")

:NdOverlay   [EXP_TYPE]
   :Scatter   [year-mon]   (AMOUNT)

In [26]:
res

EXP_TYPE,YEAR,MONTH,AMOUNT,year-mon
str,i64,i64,f64,datetime[μs]
"""Bills and Utilities""",2010,1,174.490332,2010-01-01 00:00:00
"""Bills and Utilities""",2010,2,174.938625,2010-02-01 00:00:00
"""Bills and Utilities""",2010,3,180.330367,2010-03-01 00:00:00
"""Bills and Utilities""",2010,4,182.04302,2010-04-01 00:00:00
"""Bills and Utilities""",2010,5,184.353882,2010-05-01 00:00:00
…,…,…,…,…
"""Tax""",2020,8,411.469965,2020-08-01 00:00:00
"""Tax""",2020,9,411.720836,2020-09-01 00:00:00
"""Tax""",2020,10,411.555356,2020-10-01 00:00:00


Interoperability goes in the other direction, too. We can use materialized CPU DataFrames in downstream operations on the GPU. Polars will handle any necessary CPU/GPU conversions for us, which we'll see shortly.

# Moving Beyond Our Single Dataset

Let's bring in some weather data to evaluate if transactions are impacted by weather. For example, spending behavior for certain categories could be influenced by whether it's raining or even _how much_ it's raining.

Like our transactions dataset, here we'll use a simulated rainfall dataset.

## Is more rain related to more transactions across specific categories?

Our rainfall dataset is a CSV file, so we'll use the CSV reader here and take a look at a few rows.

In this case, we'll load the data using the CPU to highlight how smoothly Polars handles mixing CPU and GPU objects.

In [27]:
names = ['Location', 'Rainfall (inches)', 'Date', 'YEAR', 'MONTH', 'DAY']

weather = pl.scan_csv(data_dir+rd, new_columns=names) # "./data/financials/rainfall_data_2010_2020.csv"
weather.head().collect()

Location,Rainfall (inches),Date,YEAR,MONTH,DAY
str,f64,i64,i64,i64,i64
"""Tatooine""",0.33,20100101,2010,1,1
"""Tatooine""",0.0,20100102,2010,1,2
"""Tatooine""",0.28,20100103,2010,1,3
"""Tatooine""",0.26,20100104,2010,1,4
"""Tatooine""",0.39,20100105,2010,1,5


Data is often messy. We've got dates in this CSV file, but they weren't parsed as dates by the CSV reader due to the format. We can easily clean this up by typecasting, after which we'll materialize the DataFrame in CPU memory.

In [28]:
weather_cleaned = (
    weather
    .with_columns(pl.col("Date").cast(pl.Utf8).str.strptime(pl.Date(), "%Y%m%d"))
    .collect()
)

To answer the original question about rainfall and spending, we need to join these datasets together.

We can pass this CPU in-memory DataFrame to a Polars Lazy API query by calling `.lazy()`.

In [29]:
%%time

(
    transactions
    .join(
        other=weather_cleaned.lazy(),
        left_on="DATE",
        right_on="Date",
        how="inner"
    )
    .group_by(["EXP_TYPE", "DATE"])
    .agg(pl.mean("Rainfall (inches)"))
    .sort(["DATE", "EXP_TYPE", "Rainfall (inches)"])
    .head()
    .collect()
)

CPU times: user 2min 24s, sys: 1min 22s, total: 3min 47s
Wall time: 8.93 s


EXP_TYPE,DATE,Rainfall (inches)
str,date,f64
"""Bills and Utilities""",2010-01-01,0.33
"""Clothing""",2010-01-01,0.33
"""Education""",2010-01-01,0.33
"""Entertainment""",2010-01-01,0.33
"""Fines""",2010-01-01,0.33


When we run the same query on the GPU, Polars handles the CPU / GPU transfers for us under the hood.

In [30]:
%%time

(
    transactions
    .join(
        other=weather_cleaned.lazy(),
        left_on="DATE",
        right_on="Date",
        how="inner"
    )
    .group_by(["EXP_TYPE", "DATE"])
    .agg(pl.mean("Rainfall (inches)"))
    .sort(["DATE", "EXP_TYPE", "Rainfall (inches)"])
    .head()
    .collect(engine=gpu_engine)
)

CPU times: user 536 ms, sys: 297 ms, total: 833 ms
Wall time: 512 ms


EXP_TYPE,DATE,Rainfall (inches)
str,date,f64
"""Bills and Utilities""",2010-01-01,0.33
"""Clothing""",2010-01-01,0.33
"""Education""",2010-01-01,0.33
"""Entertainment""",2010-01-01,0.33
"""Fines""",2010-01-01,0.33


Let's demonstrate some more of the native plotting facilities in Polars by answering the question in more holistic fashion.

We'll make a plot of total rainfall and total amount spent per month for each year and each type of expenditure.

In [31]:
res = (
    transactions
    .join(
        other=weather_cleaned.lazy(),
        left_on="DATE",
        right_on="Date",
        how="inner"
    )
    .group_by(["EXP_TYPE", "YEAR", "MONTH"])
    .agg(pl.sum("Rainfall (inches)"), pl.sum("AMOUNT"))
    .sort(["YEAR", "MONTH"])
    .collect(engine=gpu_engine)
)

res

EXP_TYPE,YEAR,MONTH,Rainfall (inches),AMOUNT
str,i64,i64,f64,f64
"""Health""",2010,1,273.21,146231.34
"""Groceries""",2010,1,1546.64,320788.47
"""Bills and Utilities""",2010,1,268.31,252138.53
"""Tax""",2010,1,67.27,197850.94
"""Education""",2010,1,105.28,134591.39
…,…,…,…,…
"""Health""",2020,12,1994.91,1.3169e7
"""Education""",2020,12,779.8,9.2533e6
"""Bills and Utilities""",2020,12,1967.79,1.7078e7


In [32]:
(
    res
    .with_columns(
        pl.date(pl.col("YEAR"), pl.col("MONTH"), 1).alias("date-month"),
        pl.col("Rainfall (inches)")*100,
    )
    .hvplot.line(
        x="date-month", y=["AMOUNT", "Rainfall (inches)"],
        by=['EXP_TYPE'],
        rot=45,
    )
)

:NdOverlay   [Variable]
   :Curve   [date-month]   (value)

We intially constructed the GPU engine with `raise_on_fail=True` to demonstrate the breadth of GPU support. Now, we'll demonstrate how a not yet GPU supported feature is handled gracefully when we set `raise_on_fail=False` (the default behavior for `engine="gpu"`).

When we use `rolling_mean`, our query smoothly falls back to the CPU to continue running rather than failing.

In [33]:
engine_with_fallback = pl.GPUEngine(
    device=0, # This is the default
    raise_on_fail=False, # Fallback to CPU if we can't run on the GPU (this is the default)
)

In [34]:
%%time

result = transactions.with_columns(
    pl.col('AMOUNT').rolling_mean(
        window_size=7
    )
     .alias('rolling_avg')
).collect(engine=engine_with_fallback)

result.head()

CPU times: user 21.3 s, sys: 1min 27s, total: 1min 48s
Wall time: 6.8 s


CUST_ID,START_DATE,END_DATE,TRANS_ID,DATE,YEAR,MONTH,DAY,EXP_TYPE,AMOUNT,rolling_avg
str,date,date,str,date,i64,i64,i64,str,f64,f64
"""CI6XLYUMQK""",2015-05-01,null,"""T8I9ZB5A6X90UG8""",2015-09-11,2015,9,11,"""Motor/Travel""",20.27,null
"""CI6XLYUMQK""",2015-05-01,null,"""TZ4JSLS7SC7FO9H""",2017-02-08,2017,2,8,"""Motor/Travel""",12.85,null
"""CI6XLYUMQK""",2015-05-01,null,"""TTUKRDDJ6B6F42H""",2015-08-01,2015,8,1,"""Housing""",383.8,null
"""CI6XLYUMQK""",2015-05-01,null,"""TDUHFRUKGPPI6HD""",2019-03-16,2019,3,16,"""Entertainment""",5.72,null
"""CI6XLYUMQK""",2015-05-01,null,"""T0JBZHBMSVRFMMD""",2015-05-15,2015,5,15,"""Entertainment""",11.06,null


# Conclusion

With the new Polars GPU engine powered by cuDF, you can potentially get significant performance gains and lower costs for workflows processing 10-100+ GB of data.

To learn more, we encourage you to visit the Polars GPU Support documentation or visit rapids.ai/cudf-polars.